In [9]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
#lista=['LINKUSDT', 'LTCUSDT', 'BCHUSDT', 'WLDUSDT', 'IDUSDT', 'HBARUSDT', 'API3USDT', 'BNBUSDT', 'C98USDT', '1000SHIBUSDT', 'STXUSDT', 'LPTUSDT', 'XLMUSDT', 'CRVUSDT', 'MKRUSDT', 'ALGOUSDT', 'HIGHUSDT', 'COMPUSDT', 'DOGEUSDT', 'XRPUSDT', 'OPUSDT', 'ARBUSDT', 'AUDIOUSDT', '1000PEPEUSDT', 'MATICUSDT', 'APEUSDT', 'SOLUSDT', 'TOMOUSDT', 'EOSUSDT', 'ADAUSDT', 'SUIUSDT', 'YGGUSDT', 'DODOXUSDT']
#lista=['MAVUSDT']

for symbol in lista:    
    try:
        data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data = md.estrategia_triangulos(symbol, tp_flag = True, print_lines_flag = True)
        #data,porcentajeentrada = md.estrategia_adrian(symbol,tp_flag = False)
        #######################################################################
        #resultado = md.backtesting(data, plot_flag = True)
        resultado = md.backtestingsanta(data, plot_flag = False)
        if resultado['Return [%]'] < 0:# and resultado['# Trades']!=0 :
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


AVAXUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
TUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan

Error:  - line: 857 - file: modulos.py


Error: 'NoneType' object is not subscriptable - line: 21 - file: 1690381842.py

GTCUSDT - Return [%]: -17.19% - # Trades: 25 - Profit Factor: 0.1476525891799635 - Win Rate [%]: 44.0
AMBUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
SANDUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
PHBUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
IMXUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
ICPUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
CHZUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
IOSTUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan

Error:  - line: 857 - file: modulos.

c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:856: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=balance)


YFIUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
AGIXUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
CFXUSDT - Return [%]: 0.53% - # Trades: 3 - Profit Factor: nan - Win Rate [%]: 100.0
OGNUSDT - Return [%]: 0.51% - # Trades: 3 - Profit Factor: nan - Win Rate [%]: 100.0
MAGICUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
NMRUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
OCEANUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
ENSUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
1INCHUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
MINAUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
TRUUSDT - Return [%]: 0.95% - # Trades: 3 - Profit Factor: nan - Win Rate [%]: 100.0

Error: local variable 'data' referenced before assignment - line: 15 - file: 169

c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:856: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=balance)


MKRUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
FILUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
UMAUSDT - Return [%]: 0.32% - # Trades: 2 - Profit Factor: nan - Win Rate [%]: 100.0
JASMYUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
STXUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
MATICUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
BAKEUSDT - Return [%]: -0.33% - # Trades: 10 - Profit Factor: 1.584443199769458 - Win Rate [%]: 70.0
SCUSDT - Return [%]: 5.79% - # Trades: 8 - Profit Factor: nan - Win Rate [%]: 100.0
XTZUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
ARKMUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan

Error:  - line: 857 - file: modulos.py


Error: 'NoneType' object is not subscriptable - line: 21 - file: 1690381842.py

LDOUSDT - Return [%]: 0.0% - 

In [ ]:
type(data)

In [ ]:
md.dibuja_patrones_triangulos (data,998) 